In [1]:
import os
import argparse
import json
import math
import random
from pathlib import Path
from collections import OrderedDict
from types import SimpleNamespace


import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2
from matplotlib import pyplot as plt


import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset


from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from scipy.ndimage import binary_fill_holes


--------------------------- Utilities ---------------------------

In [2]:
def mask_from_green_contour(annot_img, GREEN_THRESH):
    """
    Returns a binary mask where tumor area (green contour) = 1
    """
    r,g,b = cv2.split(annot_img)
    # mask = ((g < 140) & (r < 50) & ( > 190)).astype(np.uint8) #  Green
    mask = ((g < 70) & (b < 70) & (r > 150)).astype(np.uint8)  # Red

    # fill holes
    mask = cv2.dilate(mask, np.ones((3,3), np.uint8), iterations=2)
    mask = cv2.erode(mask, np.ones((3,3), np.uint8), iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8))
    mask = binary_fill_holes(mask).astype(np.uint8)
    mask = cv2.cvtColor(mask*255, cv2.COLOR_GRAY2BGR)[:,:,0] > 0

    return mask

def sample_patches(img, mask, patch_size, n_inside, n_outside, max_tries=500):
    H,W = img.shape[:2]
    inside_patches, outside_patches = [], []

    # inside
    if mask is not None:
        tries = 0
        while len(inside_patches) < n_inside and tries < max_tries:
            x = random.randint(0, W - patch_size)
            y = random.randint(0, H - patch_size)
            patch_mask = mask[y:y+patch_size, x:x+patch_size]
            # require at least 80% inside mask
            if patch_mask.mean() > 0.8:
                patch = img[y:y+patch_size, x:x+patch_size]
                inside_patches.append(patch)
            tries += 1
    else:
        mask = np.zeros((*img.shape[:2], 3), dtype=np.uint8)

    # outside
    if img is not None:
        tries = 0
        while len(outside_patches) < n_outside and tries < max_tries:
            x = random.randint(0, W - patch_size)
            y = random.randint(0, H - patch_size)
            patch_mask = mask[y:y+patch_size, x:x+patch_size]
            # require <5% overlap with tumor
            if patch_mask.mean() < 0.05 and patch_mask.mean() != 0:
                patch = img[y:y+patch_size, x:x+patch_size]
                outside_patches.append(patch)
            tries += 1

    return inside_patches, outside_patches


def load_patches_from_folders(root_dir, max_per_class=None, gray=True, resize=None):
    """Expect two folders inside root_dir: 'inside' and 'outside'."""
    inside_dir = Path(root_dir) / 'inside'
    outside_dir = Path(root_dir) / 'outside'
    os.makedirs(inside_dir, exist_ok=True)
    os.makedirs(outside_dir, exist_ok=True)

    assert inside_dir.exists() and outside_dir.exists(), "Folders 'inside' and 'outside' required"

    def load_from(folder, limit):
        files = sorted([p for p in folder.iterdir() if p.suffix.lower() in ('.tif','.jpg','.jpeg','.tif','.tiff')])
        if limit:
            files = files[:limit]
        imgs = []
        for p in files:
            im = Image.open(p)
            if gray:
                im = im.convert('L')
            else:
                im = im.convert('RGB')
            if resize:
                im = im.resize(resize, Image.BILINEAR)
            arr = np.array(im, dtype=np.float32) / 255.0
            imgs.append(arr)
        return np.stack(imgs, axis=0) if imgs else np.zeros((0, *(resize if resize else im.size[::-1])), dtype=np.float32)

    Xin = load_from(inside_dir, max_per_class)
    Xout = load_from(outside_dir, max_per_class)
    # ensure shape N,H,W
    return Xin, Xout

def to_tensor_batch(X, device):
    # X: N,H,W -> return tensor N,1,H,W
    t = torch.from_numpy(X).unsqueeze(1).to(device)
    return t

--------------------------- Kernel generators ---------------------------

In [3]:
def make_gaussian_kernel(size, sigma_x, sigma_y=None, theta=0.0):
    if sigma_y is None:
        sigma_y = sigma_x
    assert size % 2 == 1, 'size should be odd'
    half = size // 2
    xs = np.arange(-half, half+1, 1)
    ys = np.arange(-half, half+1, 1)
    X, Y = np.meshgrid(xs, ys)
    # rotate
    ct = math.cos(theta); st = math.sin(theta)
    Xr = ct * X + st * Y
    Yr = -st * X + ct * Y
    G = np.exp(-0.5 * ((Xr**2) / (sigma_x**2 + 1e-12) + (Yr**2) / (sigma_y**2 + 1e-12)))
    G = G / (G.sum() + 1e-12)
    return G.astype(np.float32)


def make_dog_kernel(size, sigma1, sigma2, theta=0.0):
    g1 = make_gaussian_kernel(size, sigma1, sigma1, theta)
    g2 = make_gaussian_kernel(size, sigma2, sigma2, theta)
    k = g1 - g2
    # normalize zero mean
    k = k - k.mean()
    return k.astype(np.float32)


def make_log_kernel(size, sigma, theta=0.0):
    # Laplacian of Gaussian approximation
    assert size % 2 == 1
    half = size // 2
    xs = np.arange(-half, half+1, 1)
    ys = np.arange(-half, half+1, 1)
    X, Y = np.meshgrid(xs, ys)
    ct = math.cos(theta); st = math.sin(theta)
    Xr = ct * X + st * Y
    Yr = -st * X + ct * Y
    r2 = (Xr**2 + Yr**2)
    s2 = sigma**2
    LoG = ((r2 - 2*s2) / (s2**2)) * np.exp(-r2/(2*s2))
    LoG = LoG - LoG.mean()
    return LoG.astype(np.float32)


def make_gabor_kernel(size, sigma, freq, theta=0.0, phase=0.0, gamma=1.0):
    # gamma: aspect ratio
    assert size % 2 == 1
    half = size // 2
    xs = np.arange(-half, half+1, 1)
    ys = np.arange(-half, half+1, 1)
    X, Y = np.meshgrid(xs, ys)
    ct = math.cos(theta); st = math.sin(theta)
    Xr = ct * X + st * Y
    Yr = -st * X + ct * Y
    Yr = Yr * gamma
    gaussian = np.exp(-(Xr**2 + Yr**2) / (2 * (sigma**2)))
    sinusoid = np.cos(2 * np.pi * freq * Xr + phase)
    K = gaussian * sinusoid
    # zero mean
    K = K - K.mean()
    # normalize by L1
    if K.sum() != 0:
        K = K / (np.abs(K).sum() + 1e-12)
    return K.astype(np.float32)


--------------------------- Sampling parameters ---------------------------

In [4]:
def sample_parameters(family, n_samples, size):
    params = []
    for _ in range(n_samples):
        if family == 'gaussian':
            sigma = float(10 ** np.random.uniform(np.log10(0.5), np.log10(size/2)))
            theta = np.random.uniform(0, math.pi)
            params.append({'sigma_x': sigma, 'sigma_y': sigma, 'theta': theta, 'size': size})
        elif family == 'anisotropic_gaussian':
            sigma_x = float(10 ** np.random.uniform(np.log10(0.5), np.log10(size/2)))
            sigma_y = float(sigma_x * np.random.uniform(0.5, 3.0))
            theta = np.random.uniform(0, math.pi)
            params.append({'sigma_x': sigma_x, 'sigma_y': sigma_y, 'theta': theta, 'size': size})
        elif family == 'dog':
            s1 = float(np.random.uniform(0.5, size/2))
            s2 = float(s1 * np.random.uniform(1.2, 3.0))
            theta = np.random.uniform(0, math.pi)
            params.append({'sigma1': s1, 'sigma2': s2, 'theta': theta, 'size': size})
        elif family == 'log':
            s = float(np.random.uniform(0.5, size/2))
            theta = np.random.uniform(0, math.pi)
            params.append({'sigma': s, 'theta': theta, 'size': size})
        elif family == 'gabor':
            sigma = float(np.random.uniform(0.5, size/2))
            freq = float(np.random.uniform(0.02, 0.5))
            theta = np.random.uniform(0, math.pi)
            phase = float(np.random.uniform(0, 2*math.pi))
            gamma = float(np.random.uniform(0.5, 1.5))
            params.append({'sigma': sigma, 'freq': freq, 'theta': theta, 'phase': phase, 'gamma': gamma, 'size': size})
        else:
            raise ValueError('Unknown family')
    return params


--------------------------- Kernel bank builder ---------------------------

In [5]:

def build_kernel_bank(families, n_per_family, size):
    bank = []
    for fam in families:
        params = sample_parameters(fam, n_per_family, size)
        for p in params:
            if fam == 'gaussian' or fam == 'anisotropic_gaussian':
                k = make_gaussian_kernel(p['size'], p['sigma_x'], p.get('sigma_y', None), p['theta'])
            elif fam == 'dog':
                k = make_dog_kernel(p['size'], p['sigma1'], p['sigma2'], p['theta'])
            elif fam == 'log':
                k = make_log_kernel(p['size'], p['sigma'], p['theta'])
            elif fam == 'gabor':
                k = make_gabor_kernel(p['size'], p['sigma'], p['freq'], p['theta'], p['phase'], p['gamma'])
            else:
                continue
            bank.append({'family': fam, 'params': p, 'kernel': k})
    return bank

--------------------------- Response computation ---------------------------

In [6]:
def compute_responses(bank, X_in, X_out, device='cpu', batch_size=64, response_fn='abs_max'):
    device = torch.device(device)
    Xin_t = to_tensor_batch(X_in, device)
    Xout_t = to_tensor_batch(X_out, device)
    responses = []
    # convert bank kernels to torch tensors (filters)
    filter_tensors = []
    for entry in bank:
        k = entry['kernel']
        k_t = torch.from_numpy(k).unsqueeze(0).unsqueeze(0).to(device)  # 1,1,Hk,Wk
        filter_tensors.append(k_t)
    # compute responses per kernel
    for idx, k_t in enumerate(tqdm(filter_tensors, desc='Kernels')):
        # conv Xin
        r_in = []
        for i in range(0, Xin_t.shape[0], batch_size):
            batch = Xin_t[i:i+batch_size]
            with torch.no_grad():
                out = F.conv2d(batch, k_t, padding=k_t.shape[-1]//2)
                if response_fn == 'abs_max':
                    val = out.abs().amax(dim=[1,2,3]).cpu().numpy()
                elif response_fn == 'mean_abs':
                    val = out.abs().mean(dim=[1,2,3]).cpu().numpy()
                else:
                    val = out.abs().amax(dim=[1,2,3]).cpu().numpy()
                r_in.append(val)
        r_in = np.concatenate(r_in, axis=0) if len(r_in) else np.zeros((0,))

        r_out = []
        for i in range(0, Xout_t.shape[0], batch_size):
            batch = Xout_t[i:i+batch_size]
            with torch.no_grad():
                out = F.conv2d(batch, k_t, padding=k_t.shape[-1]//2)
                if response_fn == 'abs_max':
                    val = out.abs().amax(dim=[1,2,3]).cpu().numpy()
                elif response_fn == 'mean_abs':
                    val = out.abs().mean(dim=[1,2,3]).cpu().numpy()
                else:
                    val = out.abs().amax(dim=[1,2,3]).cpu().numpy()
                r_out.append(val)
        r_out = np.concatenate(r_out, axis=0) if len(r_out) else np.zeros((0,))

        responses.append({'r_in': r_in, 'r_out': r_out})
    return responses


In [7]:

def fisher_score(r_in, r_out):
    mu_in = r_in.mean() if r_in.size else 0.0
    mu_out = r_out.mean() if r_out.size else 0.0
    var_in = r_in.var(ddof=1) if r_in.size>1 else 0.0
    var_out = r_out.var(ddof=1) if r_out.size>1 else 0.0
    num = (mu_in - mu_out)**2
    den = var_in + var_out + 1e-12
    return float(num / den)


def auc_score(r_in, r_out):
    y_true = np.concatenate([np.ones(len(r_in)), np.zeros(len(r_out))])
    y_score = np.concatenate([r_in, r_out])
    # if constant, AUC undefined -> return 0.5
    try:
        if y_score.max() == y_score.min():
            return 0.5
        return float(roc_auc_score(y_true, y_score))
    except Exception:
        return 0.5



In [8]:
def pairwise_response_corr(resp_i, resp_j):
    # responses already 1D arrays on combined patches
    a = resp_i - resp_i.mean()
    b = resp_j - resp_j.mean()
    denom = (np.linalg.norm(a) * np.linalg.norm(b) + 1e-12)
    return float(np.dot(a,b) / denom)


def select_diverse(bank, responses, topM=200, K=20, lambda_mm=0.75):
    # rank by combined score (we'll use AUC primarily then fisher)
    scores = []
    for i, resp in enumerate(responses):
        auc = auc_score(resp['r_in'], resp['r_out'])
        fisher = fisher_score(resp['r_in'], resp['r_out'])
        scores.append({'idx': i, 'auc': auc, 'fisher': fisher})
    scores = sorted(scores, key=lambda x: (x['auc'], x['fisher']), reverse=True)
    top_idxs = [s['idx'] for s in scores[:topM]]
    # precompute combined response vectors
    combined = []
    for i in top_idxs:
        combined.append(np.concatenate([responses[i]['r_in'], responses[i]['r_out']]))
    selected = []
    selected_idxs = []
    # greedy MMR
    for rank_i, idx in enumerate(top_idxs):
        if not selected:
            selected.append(idx)
            selected_idxs.append(idx)
            if len(selected) >= K:
                break
            continue
        # compute mmr score for remaining
        best_score = -1e9
        best_idx = None
        for j, cand in enumerate(top_idxs):
            if cand in selected_idxs:
                continue
            auc_j = next(s for s in scores if s['idx']==cand)['auc']
            # similarity to selected (max)
            sims = [abs(pairwise_response_corr(combined[top_idxs.index(cand)], combined[top_idxs.index(s)])) for s in selected]
            maxsim = max(sims) if sims else 0.0
            mmr = lambda_mm * auc_j - (1-lambda_mm) * maxsim
            if mmr > best_score:
                best_score = mmr
                best_idx = cand
        if best_idx is None:
            break
        selected.append(best_idx)
        selected_idxs.append(best_idx)
        if len(selected) >= K:
            break
    return selected_idxs


--------------------------- Training classifier ---------------------------

In [9]:
class SimpleLogistic(torch.nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.lin = torch.nn.Linear(n_features, 1)
    def forward(self, x):
        return self.lin(x).squeeze(1)


def train_classifier(Xin, Xout, selected_idxs, responses, epochs=30, batch_size=64, lr=1e-3, device='cpu'):
    # build feature matrix
    X_all = np.concatenate([Xin, Xout], axis=0)
    y = np.concatenate([np.ones(len(Xin)), np.zeros(len(Xout))])
    features = []
    for idx in selected_idxs:
        r = np.concatenate([responses[idx]['r_in'], responses[idx]['r_out']])
        features.append(r.reshape(-1,1))
    X_feat = np.concatenate(features, axis=1)
    # train/test split
    X_tr, X_val, y_tr, y_val = train_test_split(X_feat, y, test_size=0.2, stratify=y, random_state=42)
    device = torch.device(device)
    tr_ds = TensorDataset(torch.from_numpy(X_tr).float().to(device), torch.from_numpy(y_tr).float().to(device))
    val_ds = TensorDataset(torch.from_numpy(X_val).float().to(device), torch.from_numpy(y_val).float().to(device))
    tr_loader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    model = SimpleLogistic(X_feat.shape[1]).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    best_auc = 0.0
    best_state = None
    for ep in range(epochs):
        model.train()
        for xb, yb in tr_loader:
            logits = model(xb)
            loss = loss_fn(logits, yb)
            opt.zero_grad(); loss.backward(); opt.step()
        # validate
        model.eval()
        ys = []
        ps = []
        with torch.no_grad():
            for xb, yb in val_loader:
                logits = model(xb)
                probs = torch.sigmoid(logits).cpu().numpy()
                ys.append(yb.cpu().numpy())
                ps.append(probs)
        ys = np.concatenate(ys)
        ps = np.concatenate(ps)
        try:
            auc = roc_auc_score(ys, ps)
        except Exception:
            auc = 0.5
        if auc > best_auc:
            best_auc = auc
            best_state = model.state_dict()
    if best_state is not None:
        model.load_state_dict(best_state)
    # final metrics on validation
    model.eval()
    X_val_t = torch.from_numpy(X_val).float().to(device)
    with torch.no_grad():
        probs = torch.sigmoid(model(X_val_t)).cpu().numpy()
    acc = accuracy_score(y_val, (probs>0.5).astype(int))
    auc = roc_auc_score(y_val, probs)
    return {'model': model, 'auc': auc, 'acc': acc}


---------------------- Configuration ----------------------

In [10]:
IMAGE_DIR = "data/TIFF Images/all"       # folder with original mammograms
ANNOT_DIR = "data/Pixel-level annotation"  # folder with annotated images (green contour)
OUTPUT_DIR = "data/patches"          # output folder
PATCH_SIZE = 128                # size of square patches
N_INSIDE_PER_IMAGE = 50         # how many tumor patches per image
N_OUTSIDE_PER_IMAGE = 100       # healthy patches per image
GREEN_THRESH = 150              # threshold for green channel in contour
MAX_TRIES = 500                 # max attempts to sample valid patch


--------------------------- Main ---------------------------

In [15]:
annot = None
def main(args):
    device = 'cuda' if torch.cuda.is_available() and not args.force_cpu else 'cpu'
    os.makedirs(args.out_dir, exist_ok=True)

    # Make the patches directory
    output_dir = Path(OUTPUT_DIR)
    (output_dir / "inside").mkdir(parents=True, exist_ok=True)
    (output_dir / "outside").mkdir(parents=True, exist_ok=True)

    image_paths = sorted(list(Path(IMAGE_DIR).glob("*.tif")))
    annot_paths = sorted(list(Path(IMAGE_DIR).glob("*.tif")))

    inside_idx = 0
    outside_idx = 0

    # for img_path, annot_path in tqdm(zip(image_paths, annot_paths), total=len(image_paths)):
    for img_path in tqdm(image_paths, total=len(image_paths)):
        img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)

        annot = None
        if os.path.exists(str(str(Path(ANNOT_DIR) / Path(img_path.stem + '.tif')))):
            annot = cv2.imread(str(Path(ANNOT_DIR) / Path(img_path.stem + '.tif')), cv2.IMREAD_COLOR_RGB)

        mask = None
        if annot is not None:
            mask = mask_from_green_contour(annot, 125)

        inside_patches, outside_patches = sample_patches(
            img, mask, PATCH_SIZE, N_INSIDE_PER_IMAGE, N_OUTSIDE_PER_IMAGE, max_tries=MAX_TRIES * 20
        )

        # save patches
        for p in inside_patches:
            pil = Image.fromarray(p)
            pil.save(output_dir / "inside" / f"inside_{inside_idx:05d}.tif")
            inside_idx += 1

        for p in outside_patches:
            pil = Image.fromarray(p)
            pil.save(output_dir / "outside" / f"outside_{outside_idx:05d}.tif")
            outside_idx += 1

    print(f"Saved {inside_idx} inside patches and {outside_idx} outside patches to {OUTPUT_DIR}")


    # load data
    Xin, Xout = load_patches_from_folders(args.data_root, max_per_class=args.max_per_class, resize=(args.patch_size,args.patch_size))

    print(f'Loaded: in={len(Xin)} out={len(Xout)} patches. Device={device}')
    # build kernel bank
    families = args.families.split(',')
    bank = build_kernel_bank(families, args.n_per_family, args.kernel_size)
    print(f'Built kernel bank: {len(bank)} kernels')

    # compute responses
    responses = compute_responses(bank, Xin, Xout, device=device, batch_size=args.batch_size, response_fn=args.response_fn)

    # select diverse top kernels
    selected_idxs = select_diverse(bank, responses, topM=args.topM, K=args.K, lambda_mm=args.lambda_mm)
    print('Selected kernel indices:', selected_idxs)

    # train classifier
    clf_res = train_classifier(Xin, Xout, selected_idxs, responses, epochs=args.epochs, batch_size=args.batch_size, lr=args.lr, device=device)
    print(f"Classifier val AUC={clf_res['auc']:.4f} ACC={clf_res['acc']:.4f}")

    # save results
    out = {
        'selected_idxs': selected_idxs,
        'bank_meta': [{'family': b['family'], 'params': b['params']} for b in bank],
        'clf_auc': float(clf_res['auc']),
        'clf_acc': float(clf_res['acc'])
    }
    np.savez_compressed(os.path.join(args.out_dir, 'results.npz'), selected_idxs=np.array(selected_idxs))
    with open(os.path.join(args.out_dir, 'results.json'), 'w') as f:
        json.dump(out, f, indent=2)
    # save kernels
    kernels = np.stack([b['kernel'] for b in bank], axis=0)
    np.save(os.path.join(args.out_dir, 'kernels.npy'), kernels)
    print('Saved results to', args.out_dir)


if __name__ == '__main__':
    args = SimpleNamespace(
        data_root=OUTPUT_DIR,
        out_dir='./results',
        families='gaussian,anisotropic_gaussian,dog,log,gabor',
        n_per_family=200,
        kernel_size=31,
        patch_size=64,
        max_per_class=2000,
        batch_size=64,
        response_fn='abs_max',
        topM=200,
        K=20,
        lambda_mm=0.75,
        epochs=30,
        lr=1e-3,
        force_cpu=False
    )

    main(args)

100%|██████████| 511/511 [12:50<00:00,  1.51s/it]


Saved 5277 inside patches and 3220 outside patches to data/patches
Loaded: in=2000 out=2000 patches. Device=cuda
Built kernel bank: 1000 kernels


Kernels: 100%|██████████| 1000/1000 [00:57<00:00, 17.39it/s]


Selected kernel indices: [340, 852, 970, 964, 938, 315, 282, 234, 252, 346, 355, 246, 325, 60, 378, 37, 385, 128, 107, 97]
Classifier val AUC=0.9201 ACC=0.6763
Saved results to ./results
